### Usage in PyTorch

3채널짜리 64 * 64 이미지 하나를 표현하는 무작위 데이터 텐서 생성 -> 이에 상응하는 label을 무작위 값으로 초기화 

+ 3채널 = 이미지의 색상 채널. 일반적으로 3채널은 RGB 색상 모델을 의미

미리 학습된 모델의 label은 (1,1000)의 모양을 가짐

In [ ]:
import torch
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

**Forward Pass**

model에 input data를 넣어줌

In [ ]:
prediction = model(data)

**Calculating Loss**

'모델 예측값 - 정답'을 통해 오차를 계산

In [ ]:
loss = (prediction - labels).sum()

**Backward Pass**

계산한 오차를 역전파

오차 텐서에 .backward() 를 호출해 역전파 시작 

-> Autograd(파이토치의 자동 미분 엔진)가 파라미터의 .grad 속성에 모델의 각 파라미터에 대한 변화도(gradient)를 계산하고 저장

In [1]:
loss.backward()

NameError: name 'loss' is not defined

**Optimizer**

optimizer를 불러와 모델의 파라미터를 등록해줌.

learning rate = 0.01

momentum = 0.9

momentum은 SGD에서 학습을 빠르고 안정적으로 진행하기 위해 사용하는 기술. 

기본 SGD는 현재의 기울기만을 기반으로 파라미터를 업데이터하지만 momentum을 사용하면 이전 단계의 기울기를 일정 비율로 고려하여 업데이트에 반영

In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

**Step**

step을 호출하여 경사하강법을 시작. 옵티마이저는 .grad에 저장된 변화도에 따라 각 파라미터를 조정.

In [ ]:
optim.step()

### Differentiaton in Autograd

숫자 뒤에 "."찍어주는 이유 : float 자료형을 명시적으로 나타내기 위해서

"."을 찍지 않으면 int형으로 생성되는데, ML/DL 모델에서는 주로 float형을 사용한다. 정밀한 계산, 미분을 위해 필요하다.

In [ ]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

새로운 텐서 Q

In [ ]:
Q = 3*a**3 - b**2

a, b가 모두 신경망(NN)의 파라미터이고, Q는 오차이다.

1. Q에 대해 .backward()를 호출할 때, autograd는 이러한 변화도를 계산하고 이를 각 텐서의 .grad 속성에 저장.

2-1. Q가 단일 스칼라 값인 경우, backward()를 그대로 호출해주면 된다.
2-2. Q가 단일 스칼라 값이 아니라 벡터인 경우에는 방향을 알려주기 위해 Q.backward()에 gradient 인자를 명시적으로 전달해야 한다. gradient는 Q와 같은 모양의 텐서로, Q 자기 자신에 대한 gradient를 나타낸다.

In [ ]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

In [ ]:
print(9*a**2 == a.grad)
print(-2*b == b.grad)

### 연산 그래프(Computational Graph)

파이토치의 Autograd에서 사용하는 연산 그래프 

+ Autograd는 텐서 연산을 추적하여 기울기를 자동으로 계산

**Directed Acyclic Graph(DAG)**

파이토치는 DAG 형태로 연산 기록을 저장. 이 그래프는 텐서와 연산으로 구성.

리프 노드는 입력 텐서(파라미터 등), 루트 노드는 출력 텐서(손실 값 등)

**순전파**

Autograd는 순전파 시, 연산 실행하여 결과 텐서를 저장. 동시에 각 연산의 기울기 함수를 DAG에 저장. 이 기울기 함수는 이후 역전파에 사용됨.

**역전파**

루트 노드에 .backward()메서드를 호출하면 역전파 시작

Autograd는 이 루트에서 출발하여 그래프를 따라 리프 노드까지 이동하며 각 노드의 기울기를 계산 (using chain rule)

각 텐서의 .grad 속성에 계산된 기울기를 누적

In [ ]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients?: {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

In [ ]:
from torch import nn, optim

model = resnet18(weights = ResNet18_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.fc = nn.Linear(512, 10)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)